In [1]:
import streamlit as st

In [2]:
st.__version__

'1.33.0'

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
# from snowflake.connector.pandas_tools import write_pandas
# from sqlalchemy import create_engine
import snowflake.connector

from mplsoccer import Pitch, VerticalPitch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math

from snowflake.connector.pandas_tools import write_pandas

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (14.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [4]:
import snowflake.connector

In [5]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [6]:
SNOWFLAKE_USER = 'karan14'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'lv65293.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'COMPUTE_WH'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [7]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [8]:
cursor = conn.cursor()

In [9]:
def fetch_data(cursor, query):
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    return pd.DataFrame(rows, columns=column_names)

In [10]:
team_names = fetch_data(cursor, 'SELECT * FROM TEAMS')

In [11]:
team_misc = fetch_data(cursor, 'SELECT * FROM TEAM_MISC_STATS')

In [12]:
team_standard = fetch_data(cursor, 'SELECT * FROM TEAM_STANDARD_STATS')

In [13]:
team_attacking = fetch_data(cursor, 'SELECT * FROM TEAM_ATTACKING_STATS')

In [14]:
team_defending = fetch_data(cursor, 'SELECT * FROM TEAM_DEFENDING_STATS')

In [15]:
df_competitions = fetch_data(cursor, 'SELECT COMPETITION, COMPETITION_ACRONYM, SEASON FROM COMPETITIONS')

In [16]:
df_matches = fetch_data(cursor, 'SELECT * FROM MATCHES')

In [17]:
df_shots = fetch_data(cursor, 'SELECT * FROM SHOT_EVENTS')

In [18]:
df_players = fetch_data(cursor, 'SELECT * FROM PLAYERS')

In [19]:
team_attacking = team_attacking.merge(df_competitions, on=['COMPETITION','SEASON'], how='left')

team_attacking = team_attacking.merge(team_names, on='TEAM_FBREF_ID', how='left')

team_attacking = team_attacking.merge(team_standard, on=['TEAM_FBREF_ID', 'SEASON', 'COMPETITION'], how='left')

In [20]:
team_attacking['SHOTS PER GAME'] = team_attacking['SHOTS']/team_attacking['MATCHES_PLAYED']

In [21]:
team_attacking['CONVERSION RATE (%)'] = team_attacking['GOALS_SCORED']/team_attacking['SHOTS']

In [22]:
team_attacking.head()

,TEAM_FBREF_ID,SEASON,COMPETITION,GOALS_SCORED,NPXG,XG,SHOTS,SHOTS_ON_TARGET,PASS_COMPLETED,PASS_ATTEMPTED,TAKEONS_ATTEMPTED,TAKEONS_COMPLETED,CROSSES_INTO_PA,FOULS_AGAINST,COMPETITION_ACRONYM,TEAM_NAME,TEAM_LOGO_URL,TEAM_WS_ID,MATCHES_PLAYED,TEAM_WINS,TEAM_DRAWS,TEAM_LOSSES,TEAM_PTS,TEAM_XPTS,SHOTS PER GAME,CONVERSION RATE (%)
0,7a54bb4f,2223,FRA-Ligue 1,23,24.22,31.06,311,81,11216,15286,630,283,71,495,Ligue 1,Ajaccio,https://i.imgur.com/FSm4pRb.png,610,38,7,5,26,26,33.12,8.184211,0.073955
1,e2d8892c,2223,FRA-Ligue 1,89,81.59,86.15,562,242,23694,26476,976,454,46,416,Ligue 1,Paris S-G,https://i.imgur.com/kuN3QSp.png,304,38,27,4,7,85,74.01,14.789474,0.158363
2,d2c87802,2223,FRA-Ligue 1,52,42.79,47.35,397,149,14837,18087,916,406,38,501,Ligue 1,Lorient,https://i.imgur.com/9k1Kgp6.png,146,38,15,10,13,55,42.43,10.447368,0.130982
3,fb08dbb3,2223,FRA-Ligue 1,44,35.32,41.40,417,132,12486,16452,655,270,94,462,Ligue 1,Brest,https://i.imgur.com/klKUp8n.png,2332,38,11,11,16,44,43.52,10.973684,0.105516
4,7fdd64e0,2223,FRA-Ligue 1,45,53.34,59.52,516,165,13670,17565,800,347,87,423,Ligue 1,Reims,https://i.imgur.com/xULypbI.png,950,38,12,15,11,51,55.35,13.578947,0.087209


In [23]:
attacking_efficiency_chart = team_attacking[['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME', 'TEAM_LOGO_URL',
                                            'SHOTS PER GAME', 'CONVERSION RATE (%)']]

In [24]:
def upsert_to_snowflake(table_name, dataframe, primary_keys):
    if isinstance(primary_keys, str):
        primary_keys = [primary_keys]

    temp_table_name = f"{table_name}_TEMP"
    success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)

    on_condition = ' AND '.join([f"{table_name}.{pk} = {temp_table_name}.{pk}" for pk in primary_keys])

    update_columns = [col for col in dataframe.columns if col not in primary_keys]
    update_sql = ', '.join([f"{table_name}.{col} = {temp_table_name}.{col}" for col in update_columns])

    merge_sql = f"""
    MERGE INTO {table_name} USING {temp_table_name}
    ON {on_condition}
    """

    if update_columns:
        merge_sql += f"""
        WHEN MATCHED THEN
            UPDATE SET
                {update_sql}
        """

    merge_sql += f"""
    WHEN NOT MATCHED THEN
        INSERT ({', '.join(dataframe.columns)})
        VALUES ({', '.join([f"{temp_table_name}.{col}" for col in dataframe.columns])})
    """

    cur.execute(merge_sql)
    cur.execute(f"DROP TABLE IF EXISTS {temp_table_name}")

    print(f"Upsert completed. {nrows} rows processed.")

In [25]:
SNOWFLAKE_SCHEMA = 'RADAR_CHARTS'

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

In [26]:
create_table_query = """
CREATE TABLE IF NOT EXISTS TEAM_ATTACKING_EFFICIENCY_CHART (
    SEASON INT,
    COMPETITION_ACRONYM VARCHAR(255),
    TEAM_NAME VARCHAR(255),
    TEAM_LOGO_URL VARCHAR(255),
    SHOTS_PER_GAME FLOAT,
    CONVERSION_RATE FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME)
);
"""

In [27]:
cur.execute(create_table_query)

In [28]:
attacking_efficiency_chart.columns = attacking_efficiency_chart.columns.str.replace(" ", "_")

In [29]:
attacking_efficiency_chart = attacking_efficiency_chart.rename({"CONVERSION_RATE_(%)":"CONVERSION_RATE"}, axis=1)

In [30]:
attacking_efficiency_chart["CONVERSION_RATE"] = attacking_efficiency_chart["CONVERSION_RATE"] * 100

In [31]:
upsert_to_snowflake("TEAM_ATTACKING_EFFICIENCY_CHART", attacking_efficiency_chart,
                    ['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME'])

Upsert completed. 292 rows processed.


In [32]:
attacking_efficiency_chart = fetch_data(cursor, 'SELECT * FROM GEGENSTATS.RADAR_CHARTS.TEAM_ATTACKING_EFFICIENCY_CHART')

In [33]:
attacking_efficiency_chart.rename(columns={'CONVERSION_RATE':'CONVERSION RATE (%)',
                                'SHOTS_PER_GAME':'SHOTS PER GAME'}, inplace=True)

In [34]:
def create_FM_team_scatter_chart(df, chart_name, team_name, x_axis_label, y_axis_label, img_size, x_min, x_max, y_min, y_max, bottom_left_label, 
                                 bottom_right_label, top_left_label, top_right_label, bl_color, br_color, tl_color, tr_color):
    fig = go.Figure()
    x_axis_mean_val = df[x_axis_label].mean()
    y_axis_mean_val = df[y_axis_label].mean()

    # Add the scatter plot points
    for index, row in df.iterrows():
        fig.add_trace(go.Scatter(
            x=[row[x_axis_label]],
            y=[row[y_axis_label]],
            mode='markers',
            text=row["TEAM_NAME"],
            marker=dict(
                opacity=0
            ),
            hoverinfo='text',
        ))

        # Add team logo as a layout_image
        fig.add_layout_image(
            dict(
                source=row["TEAM_LOGO_URL"],
                x=row[x_axis_label],
                y=row[y_axis_label],
                xref="x",
                yref="y",
                xanchor="center",
                yanchor="middle"
            )
        )


    # Update axes and layout as necessary
    # ...
        
    fig.update_xaxes(range=[x_min, x_max], title=x_axis_label)
    fig.update_yaxes(range=[y_min, y_max], title=y_axis_label)

    fig.add_shape(
        type='line',
        x0=x_axis_mean_val, y0=fig.layout.yaxis.range[0],  # start of the line
        x1=x_axis_mean_val, y1=fig.layout.yaxis.range[1],  # end of the line
        line=dict(color='White', width=3),
        layer='below'
    )

    # Add a horizontal line at the mean aerials attempted per game
    fig.add_shape(
        type='line',
        x0=fig.layout.xaxis.range[0], y0=y_axis_mean_val,  # start of the line
        x1=fig.layout.xaxis.range[1], y1=y_axis_mean_val,  # end of the line
        line=dict(color='White', width=3),
        layer='below'
    )

    fig.update_layout(
        width=625,
        height=625,
        showlegend=False,
        paper_bgcolor='rgb(70, 70, 70)',
        plot_bgcolor='rgb(70, 70, 70)',
        font=dict(
                family="Roboto, sans-serif",  # Specify the font family
                size=25,                     # Specify the font size
                color="white"                # Specify the font color
            ),
        hoverlabel=dict(
                bgcolor="rgba(20, 20, 20, 0.8)",
                font_family="Roboto, sans-serif"),
        title={
            'text': f'{chart_name}',
            'y':0.98,  # Sets the y position of the title (1 is the top of the figure)
            'x':0.5,  # Centers the title horizontally (0.5 is the center of the figure)
            'xanchor': 'center',  # Ensures the title is centered at the x position
            'yanchor': 'top',  # Ensures the title is at the top of the y position
            'font': dict(
                family="Roboto, sans-serif",  # Specify the font family
                size=23,                     # Specify the font size
                color="white"                # Specify the font color
            )
        },
        margin=dict(l=10, r=30, t=50, b=10),
        images= [dict(
            source= row["TEAM_LOGO_URL"],
            xref="x",
            yref="y",
            x= row[x_axis_label],
            y= row[y_axis_label],
            sizex=img_size*1.8 if row['TEAM_NAME'] == team_name else img_size,  # The size of the image in x axis units
            sizey=img_size*1.8 if row['TEAM_NAME'] == team_name else img_size,  # The size of the image in y axis units
            sizing="contain",
            opacity=1 if row['TEAM_NAME'] == team_name else 0.35,
            layer="above") for index, row in df.iterrows()]
    )

    fig.update_xaxes(
        title=dict(font=dict(size=25)),
        showline=True,  # Show the axis line
        linewidth=2,  # Width of the axis line
        linecolor='white',  # Color of the axis line
        gridcolor='rgba(0,0,0,0)',  # Set grid line color to transparent
        tickfont=dict(color='white', size=15),  # Set the color of the axis ticks (numbers)
    )

    fig.update_yaxes(
        title=dict(font=dict(size=25)),
        showline=True,
        linewidth=2,
        linecolor='white',
        gridcolor='rgba(0,0,0,0)',
        tickfont=dict(color='white', size=15),
    )

    fig.add_annotation(text=bottom_left_label,
                    xref="paper", yref="paper",
                    x=0, y=0,  # Bottom left corner
                    showarrow=False,
                    font=dict(size=15, color=bl_color, family="Roboto, sans-serif"),
                    align="left")

    fig.add_annotation(text=top_left_label,
                    xref="paper", yref="paper",
                    x=0, y=1,  # Top left corner
                    showarrow=False,
                    font=dict(size=15, color=tl_color, family="Roboto, sans-serif"),
                    align="left")

    fig.add_annotation(text=top_right_label,
                    xref="paper", yref="paper",
                    x=1, y=1,  # Top right corner
                    showarrow=False,
                    font=dict(size=15, color=tr_color, family="Roboto, sans-serif"),
                    align="right")

    fig.add_annotation(text=bottom_right_label,
                    xref="paper", yref="paper",
                    x=1, y=0,  # Bottom right corner
                    showarrow=False,
                    font=dict(size=15, color=br_color, family="Roboto, sans-serif"),
                    align="right")

    return fig

In [35]:
attacking_efficiency_chart.head()

,SEASON,COMPETITION_ACRONYM,TEAM_NAME,TEAM_LOGO_URL,SHOTS PER GAME,CONVERSION RATE (%)
0,2223,Ligue 1,Ajaccio,https://i.imgur.com/FSm4pRb.png,8.184211,7.395498
1,2223,Ligue 1,Paris S-G,https://i.imgur.com/kuN3QSp.png,14.789474,15.836299
2,2223,Ligue 1,Lorient,https://i.imgur.com/9k1Kgp6.png,10.447368,13.098237
3,2223,Ligue 1,Brest,https://i.imgur.com/klKUp8n.png,10.973684,10.551559
4,2223,Ligue 1,Reims,https://i.imgur.com/xULypbI.png,13.578947,8.720930


In [36]:
season_selected = 2324
league_selected = "EPL"

In [37]:
filt_attacking_efficiency_chart = attacking_efficiency_chart[attacking_efficiency_chart['SEASON'] == season_selected]
filt_attacking_efficiency_chart = filt_attacking_efficiency_chart[filt_attacking_efficiency_chart['COMPETITION_ACRONYM'] == league_selected]

In [38]:
create_FM_team_scatter_chart(filt_attacking_efficiency_chart, 'ATTACKING EFFICIENCY', "AC Milan", 
                             'CONVERSION RATE (%)', 'SHOTS PER GAME', 0.7, 5, 20, 7, 21, 
                                                        "Passive Shooting<br>Wasteful Shooting", 
                                                        "Passive Shooting<br>Clinical Shooting",
                                                        "Aggressive Shooting<br>Wasteful Shooting",
                                                        "Aggressive Shooting<br>Clinical Shooting",
                                                        "red", "orange", "orange", "green")


In [39]:
team_attacking['GOALS PER GAME'] = team_attacking['GOALS_SCORED']/team_attacking['MATCHES_PLAYED']
team_attacking['NPXG PER GAME'] = team_attacking['NPXG']/team_attacking['MATCHES_PLAYED']

In [40]:
scoring_chart = team_attacking[['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME', 'TEAM_LOGO_URL',
                                            'GOALS PER GAME', 'NPXG PER GAME']]

In [41]:
create_table_query = """
CREATE TABLE IF NOT EXISTS TEAM_SCORING_CHART (
    SEASON INT,
    COMPETITION_ACRONYM VARCHAR(255),
    TEAM_NAME VARCHAR(255),
    TEAM_LOGO_URL VARCHAR(255),
    GOALS_PER_GAME FLOAT,
    NPXG_PER_GAME FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME)
);
"""

In [42]:
cur.execute(create_table_query)

In [43]:
scoring_chart.columns = scoring_chart.columns.str.replace(" ", "_")

In [44]:
upsert_to_snowflake("TEAM_SCORING_CHART", scoring_chart, ['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME'])

Upsert completed. 292 rows processed.


In [45]:
scoring_chart = fetch_data(cursor, 'SELECT * FROM GEGENSTATS.RADAR_CHARTS.TEAM_SCORING_CHART')

In [46]:
scoring_chart.rename(columns={'GOALS_PER_GAME':'GOALS PER GAME','NPXG_PER_GAME':'NPXG PER GAME'}, inplace=True)

In [47]:
filt_scoring_chart = scoring_chart[scoring_chart['SEASON'] == season_selected]
filt_scoring_chart = filt_scoring_chart[filt_scoring_chart['COMPETITION_ACRONYM'] == league_selected]

In [48]:
create_FM_team_scatter_chart(filt_scoring_chart, 'SCORING', "Arsenal", 
                             'NPXG PER GAME', 'GOALS PER GAME', 0.125, 0.5, 3.1, 0.5, 3.1, 
                                                        "Low Scoring<br>Low NPxG", 
                                                        "Low Scoring<br>High NPxG",
                                                        "High Scoring<br>Low NPxG",
                                                        "High Scoring<br>High NPxG",
                                                        "red", "orange", "orange", "green")


In [49]:
team_attacking.head()

,TEAM_FBREF_ID,SEASON,COMPETITION,GOALS_SCORED,NPXG,XG,SHOTS,SHOTS_ON_TARGET,PASS_COMPLETED,PASS_ATTEMPTED,TAKEONS_ATTEMPTED,TAKEONS_COMPLETED,CROSSES_INTO_PA,FOULS_AGAINST,COMPETITION_ACRONYM,TEAM_NAME,TEAM_LOGO_URL,TEAM_WS_ID,MATCHES_PLAYED,TEAM_WINS,TEAM_DRAWS,TEAM_LOSSES,TEAM_PTS,TEAM_XPTS,SHOTS PER GAME,CONVERSION RATE (%),GOALS PER GAME,NPXG PER GAME
0,7a54bb4f,2223,FRA-Ligue 1,23,24.22,31.06,311,81,11216,15286,630,283,71,495,Ligue 1,Ajaccio,https://i.imgur.com/FSm4pRb.png,610,38,7,5,26,26,33.12,8.184211,0.073955,0.605263,0.637368
1,e2d8892c,2223,FRA-Ligue 1,89,81.59,86.15,562,242,23694,26476,976,454,46,416,Ligue 1,Paris S-G,https://i.imgur.com/kuN3QSp.png,304,38,27,4,7,85,74.01,14.789474,0.158363,2.342105,2.147105
2,d2c87802,2223,FRA-Ligue 1,52,42.79,47.35,397,149,14837,18087,916,406,38,501,Ligue 1,Lorient,https://i.imgur.com/9k1Kgp6.png,146,38,15,10,13,55,42.43,10.447368,0.130982,1.368421,1.126053
3,fb08dbb3,2223,FRA-Ligue 1,44,35.32,41.40,417,132,12486,16452,655,270,94,462,Ligue 1,Brest,https://i.imgur.com/klKUp8n.png,2332,38,11,11,16,44,43.52,10.973684,0.105516,1.157895,0.929474
4,7fdd64e0,2223,FRA-Ligue 1,45,53.34,59.52,516,165,13670,17565,800,347,87,423,Ligue 1,Reims,https://i.imgur.com/xULypbI.png,950,38,12,15,11,51,55.35,13.578947,0.087209,1.184211,1.403684


In [50]:
team_attacking['SHOTS ON TARGET PER GAME'] = team_attacking['SHOTS_ON_TARGET']/team_attacking['MATCHES_PLAYED']
team_attacking['XG PER SHOT'] = team_attacking['XG']/team_attacking['SHOTS']

In [51]:
shooting_chart = team_attacking[['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME', 'TEAM_LOGO_URL',
                                            'SHOTS ON TARGET PER GAME', 'XG PER SHOT']]

In [52]:
create_table_query = """
CREATE TABLE IF NOT EXISTS TEAM_SHOOTING_CHART (
    SEASON INT,
    COMPETITION_ACRONYM VARCHAR(255),
    TEAM_NAME VARCHAR(255),
    TEAM_LOGO_URL VARCHAR(255),
    SHOTS_ON_TARGET_PER_GAME FLOAT,
    XG_PER_SHOT FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME)
);
"""

In [53]:
cur.execute(create_table_query)

In [54]:
shooting_chart.columns = shooting_chart.columns.str.replace(" ", "_")

In [55]:
upsert_to_snowflake("TEAM_SHOOTING_CHART", shooting_chart, ['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME'])

Upsert completed. 292 rows processed.


In [56]:
shooting_chart = fetch_data(cursor, 'SELECT * FROM GEGENSTATS.RADAR_CHARTS.TEAM_SHOOTING_CHART')

In [57]:
shooting_chart.rename(columns={'SHOTS_ON_TARGET_PER_GAME':'SHOTS ON TARGET PER GAME',
                                'XG_PER_SHOT':'XG PER SHOT'}, inplace=True)

In [58]:
filt_shooting_chart = shooting_chart[shooting_chart['SEASON'] == season_selected]
filt_shooting_chart = filt_shooting_chart[filt_shooting_chart['COMPETITION_ACRONYM'] == league_selected]

In [59]:
create_FM_team_scatter_chart(filt_shooting_chart, 'SHOOTING', "AC Milan", 
                             'XG PER SHOT', 'SHOTS ON TARGET PER GAME', 0.25, 0.05, 0.2, 2, 8, 
                                                        "Passive Shooting<br>Wasteful Shooting", 
                                                        "Passive Shooting<br>Clinical Shooting",
                                                        "Aggressive Shooting<br>Wasteful Shooting",
                                                        "Aggressive Shooting<br>Clinical Shooting",
                                                        "red", "orange", "orange", "green")


### Time to look at set-pieces again

In [60]:
df_events = fetch_data(cursor, 'SELECT * FROM EVENTS_SPADL')

In [61]:
df_events = df_events.merge(df_matches[['MATCH_ID','COMPETITION','SEASON']], on='MATCH_ID', how='left')

In [62]:
set_piece_cross_events = df_events[df_events['TYPE_NAME'].isin(['freekick_crossed', 'corner_crossed'])]

In [63]:
set_piece_cross_events.loc[:, 'OPPO_TEAM_ID'] = "NaN"

C:\Users\ksbha\AppData\Local\Temp\ipykernel_26560\2166643350.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [64]:
def swap_team_id(row, home_id, away_id):
    if row['TEAM_FBREF_ID'] == home_id:
        return away_id
    elif row['TEAM_FBREF_ID'] == away_id:
        return home_id
    else:
        return "PROBLEM"

In [65]:
def specify_oppo_team_id_to_events(df_events, filtered_matches):
    for ind, row in filtered_matches.iterrows():
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['AWAY_TEAM_ID']

        inds_to_replace = df_events[df_events['MATCH_ID'] == row['MATCH_ID']].index
        df_events.loc[inds_to_replace, 'OPPO_TEAM_ID'] = df_events.loc[inds_to_replace].apply(swap_team_id, 
                                                                                             axis=1, args=(home_team_id, away_team_id))
    return df_events

In [66]:
set_piece_cross_events = specify_oppo_team_id_to_events(set_piece_cross_events, df_matches)

In [67]:
team_names_2 = team_names[['TEAM_NAME', 'TEAM_FBREF_ID']].copy()
team_names_2.rename({'TEAM_FBREF_ID':'OPPO_TEAM_ID', 'TEAM_NAME':'OPPO_TEAM_NAME'}, axis=1, inplace=True)

In [68]:
set_piece_cross_events = set_piece_cross_events.merge(team_names_2, on="OPPO_TEAM_ID")

In [69]:
set_piece_cross_events = set_piece_cross_events.merge(team_names[['TEAM_NAME', 'TEAM_FBREF_ID']], on="TEAM_FBREF_ID")

In [70]:
set_piece_cross_events_grouped = set_piece_cross_events[['OPPO_TEAM_NAME','COMPETITION',
                                'SEASON','MATCH_ID']].groupby(['OPPO_TEAM_NAME','COMPETITION','SEASON']).count()

In [71]:
own_set_piece_cross_events_grouped = set_piece_cross_events[['TEAM_NAME','COMPETITION','SEASON',
                                    'MATCH_ID']].groupby(['TEAM_NAME','COMPETITION','SEASON']).count()

In [72]:
team_name_matches_played = team_attacking[['TEAM_NAME', 'COMPETITION','SEASON','MATCHES_PLAYED']].copy()

In [73]:
set_piece_cross_events_grouped.rename({'MATCH_ID':'OPPOSITION CROSSES FROM SET PIECE'}, axis=1, inplace=True)

In [74]:
own_set_piece_cross_events_grouped.rename({'MATCH_ID':'CROSSES FROM SET PIECES'}, axis=1, inplace=True)

In [75]:
set_piece_cross_events_grouped = team_name_matches_played.merge(set_piece_cross_events_grouped.reset_index().rename(
    {'OPPO_TEAM_NAME':'TEAM_NAME'}, axis=1),on=['TEAM_NAME','COMPETITION','SEASON'])

In [76]:
own_set_piece_cross_events_grouped = team_name_matches_played.merge(own_set_piece_cross_events_grouped.reset_index(),
                                                                    on=['TEAM_NAME','COMPETITION','SEASON'])

In [77]:
set_piece_box_deliveries = set_piece_cross_events.copy()

In [78]:
set_piece_box_deliveries.reset_index(drop=True, inplace=True)

In [79]:
## Only interested in crosses that end up in the box
set_piece_box_deliveries = set_piece_box_deliveries[set_piece_box_deliveries['END_X'] >= 88.5]
set_piece_box_deliveries = set_piece_box_deliveries[set_piece_box_deliveries['END_Y'] <= 54.16]
set_piece_box_deliveries = set_piece_box_deliveries[set_piece_box_deliveries['END_Y'] >= 13.84]

In [80]:
## Now, we want to cateogrize each cross as a "near post", "central", or "far post" cross
""" 
If START_Y < 30.34, then:
    - Near post is where END_Y is < 30.34
    - Central is where END_Y is between 30.34 and 37.66
    - Far post is where END_Y is > 37.66

If START_Y > 37.66, then:
    - Near post is where END_Y is > 37.66
    - Central is where END_Y is between 30.34 and 37.66
    - Far post is where END_Y is < 30.34
"""

def classify_cross(row):
    if row['START_Y'] < 34.00:
        if row['END_Y'] < 30.34:
            return 'Near post'
        elif row['END_Y'] > 37.66:
            return 'Far post'
        elif row['END_Y'] >= 30.34 and row['END_Y'] <= 37.66:
            return 'Central'
    elif row['START_Y'] >= 34.0:
        if row['END_Y'] > 37.66:
            return 'Near post'
        elif row['END_Y'] < 30.34:
            return 'Far post'
        elif row['END_Y'] >= 30.34 and row['END_Y'] <= 37.66:
            return 'Central'
    else:
        return 'Not Classified'

In [81]:
set_piece_box_deliveries['Cross_end_location'] = "NaN"
set_piece_box_deliveries['Cross_end_location'] = set_piece_box_deliveries.apply(classify_cross, axis=1)

In [82]:
set_piece_box_deliveries['Cross_end_location'].value_counts()

Cross_end_location
Central      26854
Near post    18328
Far post     12916
Name: count, dtype: int64

In [83]:
filter_1st_contacts = set_piece_box_deliveries[['MATCH_ID', 'ACTION_ID']].copy()
filter_1st_contacts['ACTION_ID'] += 1

In [84]:
set_piece_cross_1st_contacts = df_events.merge(filter_1st_contacts, on=['MATCH_ID', 'ACTION_ID'],how='right')

In [85]:
match_ids_to_remove = set_piece_cross_1st_contacts[set_piece_cross_1st_contacts['TEAM_FBREF_ID'].isna()]['MATCH_ID'].values 
action_ids_to_remove = set_piece_cross_1st_contacts[set_piece_cross_1st_contacts['TEAM_FBREF_ID'].isna()]['ACTION_ID'].values

In [86]:
## Remove missing rows!
for i in range(len(match_ids_to_remove)):
    set_piece_cross_1st_contacts = set_piece_cross_1st_contacts[(set_piece_cross_1st_contacts['MATCH_ID'] != match_ids_to_remove[i]) |
                            (set_piece_cross_1st_contacts['ACTION_ID'] != action_ids_to_remove[i])]
    
    set_piece_box_deliveries = set_piece_box_deliveries[(set_piece_box_deliveries['MATCH_ID'] != match_ids_to_remove[i]) |
                            (set_piece_box_deliveries['ACTION_ID'] != action_ids_to_remove[i]-1)]

In [87]:
set_piece_cross_1st_contacts = specify_oppo_team_id_to_events(set_piece_cross_1st_contacts, df_matches)

In [88]:
set_piece_box_deliveries = specify_oppo_team_id_to_events(set_piece_box_deliveries, df_matches)

In [89]:
set_piece_cross_1st_contacts = set_piece_cross_1st_contacts.merge(team_names[['TEAM_NAME', 'TEAM_FBREF_ID']], on="TEAM_FBREF_ID")
set_piece_cross_1st_contacts = set_piece_cross_1st_contacts.merge(team_names_2, on="OPPO_TEAM_ID")

In [90]:
# set_piece_box_deliveries = set_piece_box_deliveries.merge(team_names[['TEAM_NAME', 'TEAM_FBREF_ID']], on="TEAM_FBREF_ID")

In [91]:
set_piece_cross_1st_contacts.sort_values(by=['MATCH_ID', 'ACTION_ID'], inplace=True)
set_piece_cross_1st_contacts.reset_index(drop=True, inplace=True)
set_piece_box_deliveries.sort_values(by=['MATCH_ID', 'ACTION_ID'], inplace=True)
set_piece_box_deliveries.reset_index(drop=True, inplace=True)

In [92]:
set_piece_cross_1st_contacts['Cross_end_location'] = set_piece_box_deliveries['Cross_end_location']

In [93]:
set_piece_same_1st_contacts = set_piece_cross_1st_contacts[set_piece_box_deliveries['TEAM_FBREF_ID'] == set_piece_cross_1st_contacts['TEAM_FBREF_ID']]
set_piece_other_1st_contacts = set_piece_cross_1st_contacts[set_piece_box_deliveries['TEAM_FBREF_ID'] != set_piece_cross_1st_contacts['TEAM_FBREF_ID']]

In [94]:
att_team_won_1st_contact = set_piece_same_1st_contacts.groupby(['COMPETITION', 'SEASON', 'TEAM_NAME', 
                                                                 'Cross_end_location'])['Cross_end_location'].count()

In [95]:
att_team_lost_1st_contact = set_piece_other_1st_contacts.groupby(['COMPETITION', 'SEASON', 'OPPO_TEAM_NAME', 
                                                                 'Cross_end_location'])['Cross_end_location'].count()

In [96]:
att_team_won_1st_contact.shape

(876,)

In [97]:
att_team_lost_1st_contact.shape

(876,)

In [98]:
att_set_piece_final = (att_team_won_1st_contact/(att_team_lost_1st_contact+att_team_won_1st_contact))*100

In [99]:
att_set_piece_final = pd.DataFrame(att_set_piece_final).rename({'Cross_end_location':'PERC_1ST_CONTACT'}, axis=1).reset_index()

In [100]:
att_set_piece_final.rename(columns={'Cross_end_location':'CROSS_END_LOCATION'}, inplace=True)

In [101]:
att_set_piece_final = att_set_piece_final.merge(df_competitions, on=['COMPETITION','SEASON'])

In [102]:
att_set_piece_final.drop(['COMPETITION'], axis=1, inplace=True)

In [103]:
create_table_query = """
CREATE TABLE IF NOT EXISTS TEAM_ATT_SET_PIECE_FIRST_CONTACTS (
    SEASON INT,
    COMPETITION_ACRONYM VARCHAR(255),
    TEAM_NAME VARCHAR(255),
    PERC_1ST_CONTACT FLOAT,
    CROSS_END_LOCATION VARCHAR(255),
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME, CROSS_END_LOCATION)
);
"""

In [104]:
cur.execute(create_table_query)

In [105]:
upsert_to_snowflake("TEAM_ATT_SET_PIECE_FIRST_CONTACTS", att_set_piece_final, ['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME',
                                                                            'CROSS_END_LOCATION'])

Upsert completed. 876 rows processed.


In [106]:
att_set_piece_final = fetch_data(cursor, 'SELECT * FROM  GEGENSTATS.RADAR_CHARTS.TEAM_ATT_SET_PIECE_FIRST_CONTACTS')

In [107]:
att_set_piece_final

,SEASON,COMPETITION_ACRONYM,TEAM_NAME,PERC_1ST_CONTACT,CROSS_END_LOCATION
0,2122,EPL,Arsenal,25.892857,Central
1,2122,EPL,Arsenal,44.000000,Far post
2,2122,EPL,Arsenal,18.181818,Near post
3,2122,EPL,Aston Villa,17.948718,Central
4,2122,EPL,Aston Villa,52.000000,Far post
...,...,...,...,...,...
871,2324,Serie A,Torino,35.294118,Far post
872,2324,Serie A,Torino,36.000000,Near post
873,2324,Serie A,Udinese,47.560976,Central
874,2324,Serie A,Udinese,47.368421,Far post


In [108]:
att_set_piece_final_filt = att_set_piece_final[att_set_piece_final['SEASON'] == season_selected]
att_set_piece_final_filt = att_set_piece_final_filt[att_set_piece_final_filt['COMPETITION_ACRONYM'] == league_selected]
att_set_piece_final_filt = att_set_piece_final_filt[att_set_piece_final_filt['TEAM_NAME'] == 'Arsenal']

In [109]:
att_set_piece_chart = att_set_piece_final_filt[['CROSS_END_LOCATION', 'PERC_1ST_CONTACT']].set_index('CROSS_END_LOCATION')

In [110]:
att_set_piece_chart.loc['Near post'].values[0]

26.804123711

In [111]:
def create_set_piece_first_contacts_plot(def_set_piece_chart):
    pitch = VerticalPitch(pitch_color='#2B2B2B', line_color='white', goal_type='box', pitch_type='uefa', linewidth=1, half=True)
    fig, ax = pitch.draw(figsize=(8, 12))

    # Define the coordinates for the rectangles
    rect_coords = [
        [(13.84, 105), (13.84, 88.5), (30.09, 88.5), (30.09, 105)],  # Rect1
        [(30.59, 105), (30.59, 88.5), (37.41, 88.5), (37.41, 105)],  # Rect2
        [(37.91, 105), (37.91, 88.5), (54.16, 88.5), (54.16, 105)]  # Rect3
    ]

    # Example percentages for each rectangle (use your actual values here)
    percentages = [def_set_piece_chart.loc['Near post'].values[0], def_set_piece_chart.loc['Central'].values[0], 
                   def_set_piece_chart.loc['Far post'].values[0]]

    # Function to calculate green color based on percentage
    def get_green_color(percentage, max_percentage=100):  # Assuming 100 is the max percentage
        green_intensity = int((percentage / max_percentage) * 255)
        return f'#00{green_intensity:02x}00'

    # Create the rectangular patches
    for i, coords in enumerate(rect_coords):
        polygon = patches.Polygon(coords, closed=True, color="#00b200", zorder=2,  alpha=0.75)
        ax.add_patch(polygon)

        # Add text label in the center of each rectangle
        rect_center_x = (coords[0][0] + coords[2][0]) / 2 
        rect_center_y = (coords[0][1] + coords[2][1]) / 2
        ax.text(rect_center_x, rect_center_y, f'{int(percentages[i])}%',fontproperties='Roboto',
                va='center', ha='center', color='white', fontsize=18, zorder=3)

    # Set figure and axis background color
    fig.patch.set_facecolor('#2B2B2B')
    ax.patch.set_facecolor('#2B2B2B')

    arrow_start = (0, 107)  # Adjust these values as needed for your plot
    arrow_end = (18.84, 107)    # Adjust these values as needed for your plot

    # Draw the arrow
    ax.add_patch(patches.FancyArrow(
        arrow_start[0], arrow_start[1],  # x, y start point
        arrow_end[0] - arrow_start[0], arrow_end[1] - arrow_start[1],  # dx, dy length
        width=0.3,  # Width of the full arrow tail
        length_includes_head=False,  # The head is included in the calculation of the arrow's length
        head_width=1,  # Width of the arrow head
        head_length=1.5,  # Length of the arrow head
        color='lightgrey'  # Light grey color
    ))

    plt.title('SET PIECE FIRST CONTACTS - OPPOSITION BOX', color='gold', fontsize=20, fontname='Roboto', loc='left')

    return fig

In [112]:
create_set_piece_first_contacts_plot(att_set_piece_chart)

<Figure size 800x1200 with 1 Axes>

Shot data extraction

In [177]:
df_shots = fetch_data(cursor, 'SELECT * FROM SHOT_EVENTS')

In [178]:
df_events.head()

,MATCH_ID,ORIGINAL_EVENT_ID,PERIOD_ID,TIME_SECONDS,NEW_TIME_SECONDS,TEAM_FBREF_ID,PLAYER_WS_ID,START_X,END_X,START_Y,END_Y,RESULT_ID,ACTION_ID,TYPE_NAME,BODYPART_NAME,COMPETITION,SEASON
0,1ebfb4e4,NaN,2,654.5,534.5,98e8af82,386230,101.325,93.450,8.296,7.072,1,709,dribble,foot,ESP-La Liga,2324
1,6a69ffd2,2.673598e+09,2,749.0,629.0,0e72edf2,391047,58.695,72.450,27.200,8.636,1,940,pass,foot,ITA-Serie A,2324
2,5a35c6d1,2.672727e+09,2,1004.0,644.0,0049d422,448211,53.445,62.790,64.056,66.640,1,1049,pass,foot,ESP-La Liga,2324
3,40128bc4,2.671756e+09,1,1426.0,1426.0,e297cd13,64342,14.280,48.825,37.264,45.152,0,463,pass,foot,ENG-Premier League,2324
4,40128bc4,NaN,1,829.0,829.0,b8fd03ef,388098,94.080,102.585,55.284,51.476,1,240,dribble,foot,ENG-Premier League,2324


In [179]:
# df_shots = df_shots.merge(df_events[['MATCH_ID', 'ACTION_ID', 'TEAM_FBREF_ID']], on=['MATCH_ID', 'ACTION_ID'])
df_shots = df_shots.merge(df_events[['MATCH_ID', 'ACTION_ID', 'TEAM_FBREF_ID', 'PERIOD_ID','NEW_TIME_SECONDS','START_X','END_X',
                                     'START_Y', 'END_Y', 'PLAYER_WS_ID', 'RESULT_ID']], on=['MATCH_ID', 'ACTION_ID'])

In [180]:
df_shots = df_shots.merge(team_names[['TEAM_NAME', 'TEAM_FBREF_ID']], on="TEAM_FBREF_ID")

In [181]:
df_shots = df_shots.merge(df_matches[['MATCH_ID','HOME_TEAM_ID', 'AWAY_TEAM_ID', 'GAMEWEEK','COMPETITION',
                                      'SEASON']], on='MATCH_ID', how='left')

In [182]:
df_shots = df_shots.merge(df_competitions, on=['COMPETITION', 'SEASON'])

In [183]:
df_shots['OPPO_TEAM_ID'] = df_shots.apply(lambda row: row['AWAY_TEAM_ID'] if 
                                          row['TEAM_FBREF_ID'] == row['HOME_TEAM_ID'] else row['HOME_TEAM_ID'], axis=1)

In [184]:
df_shots.drop(['HOME_TEAM_ID', 'AWAY_TEAM_ID'], axis=1, inplace=True)

In [185]:
df_shots = df_shots.merge(team_names_2, on='OPPO_TEAM_ID')

In [186]:
df_shots = df_shots.merge(df_players[['PLAYER_WS_ID', 'PLAYER_FBREF_NAME']], on="PLAYER_WS_ID")

In [187]:
df_last_5_matches = df_matches.copy()

In [188]:
df_last_5_matches = df_last_5_matches.merge(df_competitions, on=['COMPETITION', 'SEASON'])

In [190]:
home_teams = team_names.copy()
away_teams = team_names.copy()

In [191]:
home_teams.rename({'TEAM_FBREF_ID':'HOME_TEAM_ID', 'TEAM_NAME':'HOME_TEAM_NAME'}, axis=1, inplace=True)
away_teams.rename({'TEAM_FBREF_ID':'AWAY_TEAM_ID', 'TEAM_NAME':'AWAY_TEAM_NAME'}, axis=1, inplace=True)

In [192]:
df_last_5_matches = df_last_5_matches.merge(home_teams[['HOME_TEAM_NAME', 'HOME_TEAM_ID']], on=['HOME_TEAM_ID'])
df_last_5_matches = df_last_5_matches.merge(away_teams[['AWAY_TEAM_NAME', 'AWAY_TEAM_ID']], on=['AWAY_TEAM_ID'])

In [193]:
def get_last_5_matches(group):
    return group.nlargest(5, 'GAMEWEEK')

# Splitting the original DataFrame into home and away games
home_games = df_last_5_matches[['COMPETITION', 'SEASON', 'HOME_TEAM_ID', 'GAMEWEEK']].rename(columns={'HOME_TEAM_ID': 'TEAM_ID'})
away_games = df_last_5_matches[['COMPETITION', 'SEASON', 'AWAY_TEAM_ID', 'GAMEWEEK']].rename(columns={'AWAY_TEAM_ID': 'TEAM_ID'})

# Concatenating both DataFrames to consider home and away games together
all_games = pd.concat([home_games, away_games])

# Group by 'COMPETITION', 'SEASON', and 'TEAM_ID' and apply the function to get the last 5 games
last_5_matches = all_games.groupby(['COMPETITION', 'SEASON', 'TEAM_ID']).apply(get_last_5_matches)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_26560\1360568407.py:12: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [195]:
game_weeks = [index[-1] for index in last_5_matches.index]
print(game_weeks)

[4280, 4275, 4258, 4250, 4231, 4285, 4241, 4253, 4251, 4231, 4289, 4274, 4261, 4242, 4235, 4284, 4272, 4256, 4246, 4234, 4286, 4276, 4252, 4245, 4236, 4289, 4271, 4257, 4249, 4232, 4285, 4268, 4256, 4245, 4240, 4281, 4270, 4258, 4243, 4240, 4282, 4273, 4261, 4250, 4238, 4287, 4276, 4257, 4244, 4239, 4288, 4268, 4254, 4242, 4233, 4287, 4274, 4255, 4247, 4237, 4283, 4271, 4254, 4246, 4237, 4286, 4272, 4259, 4249, 4233, 4283, 4275, 4260, 4244, 4235, 4288, 4273, 4260, 4243, 4234, 4281, 4269, 4252, 4251, 4232, 4284, 4241, 4255, 4248, 4238, 4282, 4270, 4253, 4247, 4236, 4280, 4269, 4259, 4248, 4239, 4660, 4651, 4643, 4632, 4623, 4668, 4648, 4641, 4633, 4620, 4669, 4654, 4642, 4636, 4621, 4666, 4652, 4637, 4630, 4619, 4664, 4649, 4639, 4630, 4616, 4665, 4648, 4639, 4627, 4617, 4666, 4656, 4640, 4629, 4617, 4667, 4656, 4644, 4633, 4616, 4669, 4650, 4646, 4628, 4619, 4661, 4650, 4637, 4631, 4620, 4660, 4653, 4641, 4631, 4615, 4667, 4657, 4646, 4635, 4622, 4663, 4657, 4640, 4632, 4621, 4662, 465

In [196]:
df_last_5_matches = df_last_5_matches.loc[game_weeks]

In [197]:
last_5_matches = list(df_last_5_matches['MATCH_ID'])
df_shots_last_5_matches = df_shots[df_shots['MATCH_ID'].isin(last_5_matches)]

In [198]:
df_shots_last_5_matches

,MATCH_ID,ACTION_ID,XG,PSXG,OUTCOME,DISTANCE,BODY_PART,NOTES,SCA_1_PLAYER_WS_ID,SCA_1_PLAYER_FBREF_EVENT,SCA_2_PLAYER_WS_ID,SCA_2_PLAYER_FBREF_EVENT,TEAM_FBREF_ID,PERIOD_ID,NEW_TIME_SECONDS,START_X,END_X,START_Y,END_Y,PLAYER_WS_ID,RESULT_ID,TEAM_NAME,GAMEWEEK,COMPETITION,SEASON,COMPETITION_ACRONYM,OPPO_TEAM_ID,OPPO_TEAM_NAME,PLAYER_FBREF_NAME
0,3435dfcc,113,0.13,0.19,Goal,9,Left Foot,None,332325.0,Pass (Live),238940.0,Pass (Live),18bb7c10,1,240.0,99.960,105.00,44.608,36.516,113994,1,Arsenal,29,ENG-Premier League,2324,EPL,cff3d9bb,Chelsea,Leandro Trossard
1,3435dfcc,1265,0.09,0.42,Goal,14,Left Foot,None,367185.0,Pass (Live),238940.0,Pass (Live),18bb7c10,2,1170.0,91.875,105.00,31.824,31.348,326413,1,Arsenal,29,ENG-Premier League,2324,EPL,cff3d9bb,Chelsea,Kai Havertz
2,70770e72,993,0.31,0.76,Goal,9,Right Foot,None,140667.0,Pass (Live),332143.0,Pass (Live),1df6b87e,2,249.0,96.495,105.00,35.904,35.836,332839,1,Sheffield Utd,29,ENG-Premier League,2324,EPL,19538871,Manchester Utd,Ben Brereton
3,88182e5f,1143,0.10,0.34,Goal,23,Left Foot,None,NaN,None,NaN,None,fd6114db,2,958.0,81.900,105.00,32.028,31.552,362470,1,Monaco,29,FRA-Ligue 1,2324,Ligue 1,cb188c0c,Lille,Youssouf Fofana
4,0727529c,629,0.06,NaN,Blocked,18,Left Foot,None,322668.0,Pass (Live),301869.0,Pass (Live),5725cc7b,1,1990.0,88.620,90.30,35.632,35.224,388567,0,Marseille,29,FRA-Ligue 1,2324,Ligue 1,132ebc33,Nice,Luis Henrique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130871,a370b8d5,712,0.02,NaN,Blocked,31,Right Foot,None,301243.0,Pass (Live),384143.0,Pass (Live),1c781004,1,2692.0,76.335,84.21,33.932,33.864,404572,0,Norwich City,36,ENG-Premier League,2122,EPL,7c21e445,West Ham,Jakob Sørensen
130872,a370b8d5,1193,0.05,0.03,Saved,15,Left Foot,None,79533.0,Pass (Live),79533.0,Take-On,1c781004,2,1555.0,92.400,103.32,45.764,36.176,377190,0,Norwich City,36,ENG-Premier League,2122,EPL,7c21e445,West Ham,Przemysław Płacheta
130873,a370b8d5,1425,0.06,NaN,Off Target,29,Right Foot,Free kick,377190.0,Fouled,384143.0,Pass (Live),1c781004,2,2297.0,78.330,105.00,40.052,37.468,381255,0,Norwich City,36,ENG-Premier League,2122,EPL,7c21e445,West Ham,Billy Gilmour
130874,a370b8d5,1452,0.05,NaN,Off Target,25,Right Foot,None,377190.0,Pass (Live),381255.0,Pass (Live),1c781004,2,2379.0,82.005,105.00,39.440,29.172,25028,0,Norwich City,36,ENG-Premier League,2122,EPL,7c21e445,West Ham,Teemu Pukki


In [203]:
create_table_query = """
CREATE TABLE IF NOT EXISTS SHOT_MAP (
    MATCH_ID VARCHAR(255),
    ACTION_ID INT,
    XG DOUBLE,
    PSXG DOUBLE,
    OUTCOME VARCHAR(255),
    DISTANCE INT,
    BODY_PART VARCHAR(255),
    NOTES VARCHAR(255),
    SCA_1_PLAYER_WS_ID INT,
    SCA_1_PLAYER_FBREF_EVENT VARCHAR(255),
    SCA_2_PLAYER_WS_ID INT,
    SCA_2_PLAYER_FBREF_EVENT VARCHAR(255),
    TEAM_FBREF_ID VARCHAR(255),
    PERIOD_ID INT,
    NEW_TIME_SECONDS DOUBLE,
    START_X DOUBLE, 
    END_X DOUBLE,
    START_Y DOUBLE,
    END_Y DOUBLE,
    PLAYER_WS_ID INT,
    RESULT_ID INT,
    TEAM_NAME VARCHAR(255),
    GAMEWEEK INT,
    COMPETITION VARCHAR(255),
    SEASON INT,
    COMPETITION_ACRONYM VARCHAR(255),
    OPPO_TEAM_ID VARCHAR(255),
    OPPO_TEAM_NAME VARCHAR(255),
    PLAYER_FBREF_NAME VARCHAR(255),
    PRIMARY KEY (MATCH_ID, ACTION_ID),
    FOREIGN KEY (MATCH_ID) REFERENCES GEGENSTATS.TABLES.MATCHES(MATCH_ID),
    FOREIGN KEY (MATCH_ID, ACTION_ID) REFERENCES GEGENSTATS.TABLES.SHOT_EVENTS(MATCH_ID, ACTION_ID),
    FOREIGN KEY (SCA_1_PLAYER_WS_ID) REFERENCES GEGENSTATS.TABLES.PLAYERS(PLAYER_WS_ID),
    FOREIGN KEY (SCA_2_PLAYER_WS_ID) REFERENCES GEGENSTATS.TABLES.PLAYERS(PLAYER_WS_ID)
);
"""

In [204]:
cur.execute(create_table_query)

In [205]:
upsert_to_snowflake("SHOT_MAP", df_shots_last_5_matches, ['MATCH_ID', 'ACTION_ID'])

Upsert completed. 18966 rows processed.


In [238]:
df_shots_last_5_matches = fetch_data(cursor, 'SELECT * FROM  GEGENSTATS.RADAR_CHARTS.SHOT_MAP')

In [239]:
# df_shots = df_shots.merge(df_events[['MATCH_ID', 'ACTION_ID', 'PERIOD_ID','NEW_TIME_SECONDS','START_X','END_X','START_Y', 
#                          'END_Y', 'PLAYER_WS_ID', 'RESULT_ID']], on=['MATCH_ID', 'ACTION_ID'])

In [240]:
# df_shots = df_shots.merge(df_matches[['MATCH_ID', 'HOME_TEAM_ID', 'AWAY_TEAM_ID', 'GAMEWEEK']], on='MATCH_ID', how='left')

In [241]:
# df_shots['OPPO_TEAM_ID'] = df_shots.apply(lambda row: row['AWAY_TEAM_ID'] if row['TEAM_FBREF_ID'] == row['HOME_TEAM_ID'] else row['HOME_TEAM_ID'], axis=1)

In [242]:
# df_shots.drop(['HOME_TEAM_ID', 'AWAY_TEAM_ID'], axis=1, inplace=True)

In [243]:
# df_shots = df_shots.merge(team_names_2, on='OPPO_TEAM_ID')

In [244]:
# df_shots = df_shots.merge(df_players[['PLAYER_WS_ID', 'PLAYER_FBREF_NAME']], on="PLAYER_WS_ID")

In [245]:
team_selected = 'Nantes'

In [246]:
# df_last_5_matches_filt = df_last_5_matches_filt[(df_last_5_matches_filt['HOME_TEAM_NAME'] == team_selected) | 
#                                                 (df_last_5_matches_filt['AWAY_TEAM_NAME'] == team_selected)]
# df_last_5_matches_filt = df_last_5_matches_filt.sort_values(by='GAMEWEEK')[-5:]
# last_5_matches = list(df_last_5_matches_filt['MATCH_ID'])
# df_shots_last_5_matches = df_shots[df_shots['MATCH_ID'].isin(last_5_matches)]

In [249]:
df_shots_last_5_matches = df_shots_last_5_matches[df_shots_last_5_matches['SEASON'] == 2223]
df_shots_last_5_matches = df_shots_last_5_matches[df_shots_last_5_matches['COMPETITION_ACRONYM'] == 'Ligue 1']

In [250]:
df_shots_last_5_matches = df_shots_last_5_matches[df_shots_last_5_matches['TEAM_NAME'] == team_selected]

In [251]:
df_shots_last_5_matches['OUTCOME'].value_counts()

OUTCOME
Off Target          19
Saved               17
Blocked             13
Goal                 2
Woodwork             1
Saved off Target     1
Name: count, dtype: int64

In [252]:
df_shots_last_5_matches['norm_start_x'] = df_shots_last_5_matches['START_X'] / 120
df_shots_last_5_matches['norm_start_y'] = df_shots_last_5_matches['START_Y'] / 80

In [253]:
max_size = 5
min_size = 2

In [254]:
xg_scaled = (df_shots_last_5_matches['XG'] - df_shots_last_5_matches['XG'].min()) / (df_shots_last_5_matches['XG'].max() - df_shots_last_5_matches['XG'].min())
df_shots_last_5_matches['xG_size'] = xg_scaled * (max_size - min_size) + min_size

In [255]:
summary_data = df_shots_last_5_matches['OUTCOME'].value_counts().reset_index()
summary_data.columns = ['Outcome', 'Count']

In [256]:
total_xg = df_shots_last_5_matches['XG'].sum()
Goals = df_shots_last_5_matches['OUTCOME'].value_counts()['Goal']
Attempts = df_shots_last_5_matches.shape[0]
with_feet = df_shots_last_5_matches['BODY_PART'].str.contains('Foot', na=False).sum()
with_head = df_shots_last_5_matches['BODY_PART'].value_counts()['Head']
direct_set_pieces = df_shots_last_5_matches['NOTES'].str.contains('Free kick', na=False).sum()

In [257]:
total_xg, Goals, Attempts, with_feet, with_head, direct_set_pieces

(4.98, 2, 53, 45, 8, 3)

In [261]:
import plotly.graph_objects as go


df_shots_last_5_matches['hover_text'] = (
    '<span style="font-size: 20px; line-height: 30px;"><b>' + df_shots_last_5_matches['PLAYER_FBREF_NAME'].astype(str) + '</b></span>' +
    '<span style="font-size: 12px; line-height: 22px;">  vs ' + df_shots_last_5_matches['OPPO_TEAM_NAME'].astype(str) + '</span>'
    '<br><span style="font-size: 15px; line-height: 18px;">Shot - ' + df_shots_last_5_matches['OUTCOME'].astype(str) + '</span>' +
    '<br><span style="font-size: 15px; line-height: 18px;">Shot xG - ' + df_shots_last_5_matches['XG'].astype(str) + '</span>' +
    '<br><span style="font-size: 15px; line-height: 18px;">Game time - ' + ((df_shots_last_5_matches['NEW_TIME_SECONDS']//60 + 1).astype(int)).astype(str) + "'"+ 
    '<span style="font-size: 12px; line-height: 22px;">  Period/Half: ' + df_shots_last_5_matches['PERIOD_ID'].astype(str) + '</span>'
)

outcome_colors = {'Goal': 'forestgreen', 'Saved': 'red', 'Blocked': 'orange', 'Off Target': 'red', 
                  'Woodwork': 'purple', 'Saved off Target': 'red'}
outcome_markers = {'Goal': 'o', 'Saved': 'o', 'Blocked': 'X', 'Off Target': 'X',  'Woodwork': 'h',
                   'Saved off Target': 'X'}
outcome_alpha = {'Goal': .8, 'Saved': 0.6, 'Blocked': 0.6, 'Off Target': 0.5,  'Woodwork': 0.7,
                 'Saved off Target': 0.5}

# Create a Plotly figure
fig = go.Figure()

# Add the pitch as a background image
fig.add_layout_image(
    dict(
        source="https://i.imgur.com/L0GXGh5.png",  # Path to your pitch background image
        xref="x",
        yref="y",
        x=0.,
        y=110,
        sizex=60,
        sizey=95,
        # sizing="stretch",
        opacity=1.0,
        layer="below")
)

# Define marker symbols in Plotly's format
plotly_symbols = {
    'Goal': 'circle',
    'Saved': 'circle',
    'Blocked': 'x',
    'Off Target': 'x',
    'Woodwork': 'hexagon',
    'Saved off Target': 'x'
}

# header_values = []  # No header values, since we're incorporating the title directly in the cell
header_values = [
    ['<span style="font-size: 10px; color: grey;">EXPECTED GOALS</span>', 
     '<span style="font-size: 10px; color: grey;">GOALS</span>',
     '<span style="font-size: 10px; color: grey;">ATTEMPTS</span>',
     '<span style="font-size: 10px; color: grey;">WITH FEET</span>', 
     '<span style="font-size: 10px; color: grey;">WITH HEAD</span>',
     '<span style="font-size: 10px; color: grey;">DIRECT SET PIECES</span>',
     # ... add other outcomes as needed
    '<span style="font-size: 22px; color: white;"><b>total_xg</b></span>',
     '<span style="font-size: 22px; color: white;"><b>Goals</b></span>',
     '<span style="font-size: 22px; color: white;"><b>Attempts</b></span>',
     '<span style="font-size: 22px; color: white;"><b>with_feet</b></span>',
     '<span style="font-size: 22px; color: white;"><b>with_head</b></span>',
     '<span style="font-size: 22px; color: white;"><b>direct_set_pieces</b></span>',
    ]
]

# Add scatter plots for each outcome type
for outcome, df_group in df_shots_last_5_matches.groupby('OUTCOME'):
    fig.add_trace(go.Scatter(
        x=(df_group['START_Y'] - 68)*-0.8825,  # Assuming START_Y is the horizontal axis in your pitch image
        y=df_group['START_X']+0.25,  # Assuming START_X is the vertical axis in your pitch image
        mode='markers',
        marker=dict(
            color=outcome_colors[outcome],
            symbol=plotly_symbols[outcome],
            size=df_group['xG_size']*5,  # Adjust size scaling factor as needed
            opacity=df_group['OUTCOME'].map(outcome_alpha)
        ),
        name=outcome,
        text=df_group['hover_text'],  # This will be displayed on hover
        hoverinfo='text',
        # hovertemplate='<b>%{text}</b>'  # Custom hover template for cleaner text display
    ))

# Set axes to match the background image
fig.update_xaxes(showgrid=False, zeroline=False, visible=False, range=[0, 68])
fig.update_yaxes(showgrid=False, zeroline=False, visible=False, range=[52.5, 110])

# Remove the plot background color
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                    font=dict(
                            family="Calibri, sans-serif",  # Set the font to Calibri, with sans-serif as a fallback
                            size=12,  # You can adjust the base size as needed
                            color="black"  # And also set a global font color if you wish
                        ), width=1150,height=700)

fig.add_trace(go.Table(
    header=dict(
        values=[
            ["" ,"<b>TOTAL xG</b>", "", "<b>GOALS</b>", "", "<b>ATTEMPTS</b>", "", "<b>W/ FEET</b>", "", "<b>W/ HEAD</b>", "","<b>DIRECT FKs</b>"],
            ["", f"{round(total_xg, 1)}", "", f"{Goals}", "", f"{Attempts}", "", f"{with_feet}", "", f"{with_head}", "", f"{direct_set_pieces}"]
        ],
        line_color='#2B2B2B',
        fill_color='#2B2B2B',
        align=['right', 'left'],  # Align each column differently if needed
        font=dict(color=['grey', 'white'], size=[18, 28])  # Specify different styles for each column
    ),
    domain=dict(x=[0.835, 1], y=[0, 1])
))

fig.update_layout(
    paper_bgcolor="#2B2B2B",
    plot_bgcolor="#2B2B2B",
    margin=dict(l=0, r=22, t=0, b=0),
    legend=dict(
        orientation="h",  # Set the legend orientation to horizontal
        yanchor="bottom",
        y=-0.03,  # Negative value to move the legend below the plot
        xanchor="center",
        x=0.45,
        font=dict(  # Update the font size
            size=25,  # Example size, adjust as needed
            color="white"  # Set the text color to white for better contrast on a dark background
        ),
        bgcolor="#2B2B2B",  # Set the background color of the legend
        itemsizing='constant'  # Use the same size for all legend markers
    ),
    title={
            'text': 'SHOT MAP',
            'y':0.97,  # Sets the y position of the title (1 is the top of the figure)
            'x':0.1,  # Centers the title horizontally (0.5 is the center of the figure)
            'xanchor': 'center',  # Ensures the title is centered at the x position
            'yanchor': 'top',  # Ensures the title is at the top of the y position
            'font': dict(
                family="Roboto",  # Specify the font family
                size=23,                     # Specify the font size
                color="gold"                # Specify the font color
            )
        }
)

fig.show()

In [259]:
df_shots_last_5_matches

,MATCH_ID,ACTION_ID,XG,PSXG,OUTCOME,DISTANCE,BODY_PART,NOTES,SCA_1_PLAYER_WS_ID,SCA_1_PLAYER_FBREF_EVENT,SCA_2_PLAYER_WS_ID,SCA_2_PLAYER_FBREF_EVENT,TEAM_FBREF_ID,PERIOD_ID,NEW_TIME_SECONDS,START_X,END_X,START_Y,END_Y,PLAYER_WS_ID,RESULT_ID,TEAM_NAME,GAMEWEEK,COMPETITION,SEASON,COMPETITION_ACRONYM,OPPO_TEAM_ID,OPPO_TEAM_NAME,PLAYER_FBREF_NAME,norm_start_x,norm_start_y,xG_size,hover_text
6341,1b4970df,1678,0.12,0.20,Saved,16,Right Foot,None,476406.0,Pass (Live),476406.0,Take-On,d7a486cd,2,3022.0,90.510,103.215,28.152,31.416,202185,0,Nantes,37,FRA-Ligue 1,2223,Ligue 1,cb188c0c,Lille,Moses Simon,0.754250,0.35190,2.523810,"<span style=""font-size: 20px; line-height: 30p..."
6351,8411b267,584,0.09,NaN,Off Target,18,Right Foot,None,411087.0,Pass (Live),411087.0,Take-On,d7a486cd,1,1871.0,88.200,105.000,25.092,30.872,294496,0,Nantes,34,FRA-Ligue 1,2223,Ligue 1,c0d3eab4,Strasbourg,Fabien Centonze,0.735000,0.31365,2.380952,"<span style=""font-size: 20px; line-height: 30p..."
6357,54a0b007,262,0.64,0.75,Goal,7,Right Foot,None,299562.0,Take-On,299562.0,Take-On,d7a486cd,1,902.0,98.700,105.000,32.980,33.388,344645,1,Nantes,38,FRA-Ligue 1,2223,Ligue 1,69236f98,Angers,Ignatius Ganago,0.822500,0.41225,5.000000,"<span style=""font-size: 20px; line-height: 30p..."
6368,1ed26dbe,1313,0.11,NaN,Off Target,4,Left Foot,Volley,279380.0,Pass (Live),114169.0,Pass (Live),d7a486cd,2,1116.0,102.585,105.000,27.812,33.048,299562,0,Nantes,36,FRA-Ligue 1,2223,Ligue 1,281b0e73,Montpellier,Ludovic Blas,0.854875,0.34765,2.476190,"<span style=""font-size: 20px; line-height: 30p..."
6391,52ee44a2,508,0.18,NaN,Woodwork,13,Left Foot,None,114169.0,Pass (Live),256539.0,Pass (Live),d7a486cd,1,1278.0,93.135,105.000,38.760,34.748,342571,0,Nantes,35,FRA-Ligue 1,2223,Ligue 1,3f8c4b5f,Toulouse,Samuel Moutoussamy,0.776125,0.48450,2.809524,"<span style=""font-size: 20px; line-height: 30p..."
6397,52ee44a2,953,0.01,NaN,Blocked,33,Left Foot,None,256539.0,Take-On,256539.0,Take-On,d7a486cd,1,2627.0,74.655,76.545,25.228,27.336,299562,0,Nantes,35,FRA-Ligue 1,2223,Ligue 1,3f8c4b5f,Toulouse,Ludovic Blas,0.622125,0.31535,2.000000,"<span style=""font-size: 20px; line-height: 30p..."
6425,8411b267,737,0.04,NaN,Blocked,8,Head,None,299562.0,Pass (Dead),NaN,None,d7a486cd,1,2361.0,98.805,101.745,41.616,39.304,279380,0,Nantes,34,FRA-Ligue 1,2223,Ligue 1,c0d3eab4,Strasbourg,Andrei Girotto,0.823375,0.52020,2.142857,"<span style=""font-size: 20px; line-height: 30p..."
6429,52ee44a2,2049,0.15,NaN,Off Target,6,Head,None,299562.0,Pass (Dead),29595.0,Pass (Live),d7a486cd,2,2876.0,99.225,105.000,31.960,37.128,279380,0,Nantes,35,FRA-Ligue 1,2223,Ligue 1,3f8c4b5f,Toulouse,Andrei Girotto,0.826875,0.39950,2.666667,"<span style=""font-size: 20px; line-height: 30p..."
6451,54a0b007,3,0.03,NaN,Off Target,21,Right Foot,Volley,NaN,None,NaN,None,d7a486cd,1,7.0,87.780,16.170,18.768,47.056,344645,0,Nantes,38,FRA-Ligue 1,2223,Ligue 1,69236f98,Angers,Ignatius Ganago,0.731500,0.23460,2.095238,"<span style=""font-size: 20px; line-height: 30p..."
6463,1ed26dbe,1545,0.04,NaN,Saved off Target,12,Right Foot,None,294496.0,Pass (Live),342571.0,Pass (Live),d7a486cd,2,1830.0,101.640,103.740,19.788,32.164,299562,0,Nantes,36,FRA-Ligue 1,2223,Ligue 1,281b0e73,Montpellier,Ludovic Blas,0.847000,0.24735,2.142857,"<span style=""font-size: 20px; line-height: 30p..."


In [150]:
df_shots['BODY_PART'].value_counts()

BODY_PART
Right Foot    64869
Left Foot     42290
Head          23181
Other           557
Name: count, dtype: int64

In [151]:
df_shots['NOTES'].value_counts()

NOTES
Volley                  15963
Free kick                4280
Deflected                2998
Deflected, Volley         274
Free kick, Deflected       89
nan                         6
Free kick, Volley           1
Name: count, dtype: int64

In [152]:
df_shots['OUTCOME'].value_counts()

OUTCOME
Off Target          48646
Blocked             35116
Saved               30450
Goal                14046
Woodwork             2507
Saved off Target      132
Name: count, dtype: int64

In [153]:
df_shots = fetch_data(cursor, 'SELECT * FROM SHOT_EVENTS')
df_shots = df_shots.merge(df_events[['MATCH_ID', 'ACTION_ID', 'TEAM_FBREF_ID']], on=['MATCH_ID', 'ACTION_ID'])
df_shots = df_shots.merge(team_names[['TEAM_NAME', 'TEAM_FBREF_ID']], on="TEAM_FBREF_ID")

inds1 = df_shots[df_shots['SCA_1_PLAYER_FBREF_EVENT'].isin(['Pass (Dead)'])].index
inds2 = df_shots[df_shots['SCA_2_PLAYER_FBREF_EVENT'].isin(['Pass (Dead)'])].index

inds_union = np.union1d(inds1, inds2)
df_shots_set_piece = df_shots.loc[inds_union]

df_shots_set_piece = df_events[['MATCH_ID','ACTION_ID']].merge(df_shots_set_piece, on=['MATCH_ID', 'ACTION_ID'])
df_shots_set_piece.loc[:, 'OPPO_TEAM_ID'] = "NaN"
df_shots_set_piece = specify_oppo_team_id_to_events(df_shots_set_piece, df_matches)
df_shots_set_piece = df_shots_set_piece.merge(team_names_2, on="OPPO_TEAM_ID")
df_shots_set_piece = df_shots_set_piece.merge(df_matches[['MATCH_ID','COMPETITION','SEASON']], on='MATCH_ID', how='left')

In [154]:
shots_xg_conceded_from_set_pieces_grouped = df_shots_set_piece[['OPPO_TEAM_NAME','COMPETITION','SEASON', 
                                                                'XG']].groupby(['OPPO_TEAM_NAME','COMPETITION','SEASON']).sum()

In [155]:
shots_xg_from_set_pieces_grouped = df_shots_set_piece[['TEAM_NAME','COMPETITION','SEASON', 
                                                    'XG']].groupby(['TEAM_NAME','COMPETITION','SEASON']).sum()

In [156]:
shots_xg_conceded_from_set_pieces_grouped = team_name_matches_played.merge(shots_xg_conceded_from_set_pieces_grouped.reset_index().rename(
    {'OPPO_TEAM_NAME':'TEAM_NAME'}, axis=1),on=['TEAM_NAME','COMPETITION','SEASON'])

In [157]:
shots_xg_from_set_pieces_grouped = team_name_matches_played.merge(shots_xg_from_set_pieces_grouped.reset_index(),
                                                                  on=['TEAM_NAME','COMPETITION','SEASON'])

In [158]:
shots_xg_conceded_from_set_pieces_grouped['OPPOSITION XG FROM SET PIECE CROSSES PER GAME'] = shots_xg_conceded_from_set_pieces_grouped['XG'] / \
                                                                        shots_xg_conceded_from_set_pieces_grouped['MATCHES_PLAYED']

In [159]:
shots_xg_from_set_pieces_grouped['XG FROM SET PIECE CROSSES PER GAME'] = shots_xg_from_set_pieces_grouped['XG'] / \
                                                                        shots_xg_from_set_pieces_grouped['MATCHES_PLAYED']

In [160]:
# shots_xg_from_set_pieces_grouped
# shots_xg_conceded_from_set_pieces_grouped

In [161]:
df_goals_set_piece = df_shots_set_piece[df_shots_set_piece['OUTCOME'] == 'Goal']

In [162]:
df_goals_set_piece_chart = df_goals_set_piece.groupby(['TEAM_NAME','COMPETITION','SEASON']).count()['MATCH_ID']

In [163]:
df_goals_set_piece_chart = df_goals_set_piece_chart.reset_index()
df_goals_set_piece_chart = df_goals_set_piece_chart.rename({'MATCH_ID':'SET PIECE GOALS SCORED'}, axis= 1)

In [164]:
df_goals_set_piece_conc = df_goals_set_piece.groupby(['OPPO_TEAM_NAME','COMPETITION','SEASON']).count()['MATCH_ID']
df_goals_set_piece_conc = df_goals_set_piece_conc.reset_index()
df_goals_set_piece_conc = df_goals_set_piece_conc.rename({'MATCH_ID':'SET PIECE GOALS CONCEDED',
                                                            'OPPO_TEAM_NAME':'TEAM_NAME'}, axis= 1)

In [165]:
df_goals_set_piece_chart = df_goals_set_piece_chart.merge(df_goals_set_piece_conc, on=['TEAM_NAME', 'COMPETITION', 'SEASON'], how='outer')

In [166]:
df_goals_set_piece_chart = df_goals_set_piece_chart.fillna(0)

In [167]:
df_goals_set_piece_chart = df_goals_set_piece_chart.merge(team_names[['TEAM_NAME', 'TEAM_LOGO_URL']], on='TEAM_NAME')
df_goals_set_piece_chart = df_goals_set_piece_chart.merge(df_competitions, on=['COMPETITION', 'SEASON'])

In [168]:
create_table_query = """
CREATE TABLE IF NOT EXISTS GOALS_FROM_SET_PIECES (
    SEASON INT,
    COMPETITION_ACRONYM VARCHAR(255),
    TEAM_NAME VARCHAR(255),
    TEAM_LOGO_URL VARCHAR(255),
    SET_PIECE_GOALS_SCORED FLOAT,
    SET_PIECE_GOALS_CONCEDED FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME)
);
"""

In [169]:
cur.execute(create_table_query)

In [170]:
df_goals_set_piece_chart.columns = df_goals_set_piece_chart.columns.str.replace(" ", "_")

In [171]:
df_goals_set_piece_chart = df_goals_set_piece_chart.drop('COMPETITION', axis=1)

In [172]:
upsert_to_snowflake("GOALS_FROM_SET_PIECES", df_goals_set_piece_chart, ['SEASON', 'COMPETITION_ACRONYM', 'TEAM_NAME'])

Upsert completed. 292 rows processed.


In [173]:
df_goals_set_piece_chart = fetch_data(cursor, 'SELECT * FROM GEGENSTATS.RADAR_CHARTS.GOALS_FROM_SET_PIECES')

In [174]:
df_goals_set_piece_chart.rename(columns={'SET_PIECE_GOALS_SCORED':'SET PIECE GOALS SCORED',
                                'SET_PIECE_GOALS_CONCEDED':'SET PIECE GOALS CONCEDED'}, inplace=True)

In [175]:
filt_df_goals_set_piece_chart = df_goals_set_piece_chart[df_goals_set_piece_chart['SEASON'] == season_selected]
filt_df_goals_set_piece_chart = filt_df_goals_set_piece_chart[filt_df_goals_set_piece_chart['COMPETITION_ACRONYM'] == league_selected]

In [176]:
create_FM_team_scatter_chart(filt_df_goals_set_piece_chart, 'GOALS FROM SET PIECES', "AC Milan", 
                             'SET PIECE GOALS CONCEDED', 'SET PIECE GOALS SCORED', 0.9, 0, 16, 0, 17, 
                                                        "Low no. of set piece goals<br>Low no. of set piece goals conceded", 
                                                        "Low no. of set piece goals<br>High no. of set piece goals conceded",
                                                        "High no. of set piece goals<br>Low no. of set piece goals conceded",
                                                        "High no. of set piece goals<br>High no. of set piece goals conceded",
                                                        "orange", "red", "green", "orange")
